<a href="https://colab.research.google.com/github/HyeonhoonLee/marbling/blob/master/animals_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Classifying either beaver, panda, Leopards, or dalmatian**

In [ ]:
from PIL import Image
import os, glob
import numpy as np
import random, math
# 분류 대상 카테고리
root_dir = "/content/drive/My Drive/WikiExample/ch7/image/101_ObjectCategories"
categories = ["beaver", "panda", "Leopards", "dalmatian"]
nb_classes = len(categories)
image_size = 50
# 이미지 데이터 읽어 들이기 --- (※1)
X = [] # 이미지 데이터
Y = [] # 레이블 데이터
def add_sample(cat, fname, is_train):
    img = Image.open(fname)
    img = img.convert("RGB") # 색상 모드 변경하기
    img = img.resize((image_size, image_size)) # 이미지 크기 변경하기
    data = np.asarray(img)
    X.append(data)
    Y.append(cat)
    if not is_train: return
    # 각도를 조금 변경한 파일 추가하기
    # 회전하기
    for ang in range(-20, 20, 5):
        img2 = img.rotate(ang)
        data = np.asarray(img2)
        X.append(data)
        Y.append(cat)
        # img2.save("gyudon-"+str(ang)+".PNG")
        # 반전하기
        img2 = img2.transpose(Image.FLIP_LEFT_RIGHT)
        data = np.asarray(img2)
        X.append(data)
        Y.append(cat)
def make_sample(files, is_train):
    global X, Y
    X = []; Y = []
    for cat, fname in files:
        add_sample(cat, fname, is_train)
    return np.array(X), np.array(Y)
# 각 폴더에 들어있는 파일 수집하기 --- (※2)
allfiles = []
for idx, cat in enumerate(categories):
    image_dir = root_dir + "/" + cat
    files = glob.glob(image_dir + "/*.jpg")
    for f in files:
        allfiles.append((idx, f))
# 섞은 뒤에 학습 전용 데이터와 테스트 전용 데이터 구분하기 --- (※3)
random.shuffle(allfiles)
th = math.floor(len(allfiles) * 0.6)
train = allfiles[0:th]
test  = allfiles[th:]
X_train, y_train = make_sample(train, True)
X_test, y_test = make_sample(test, False)
xy = (X_train, X_test, y_train, y_test)
np.save("/content/drive/My Drive/WikiExample/ch7/image/animals.npy", xy)
print("ok,", len(y_train))

ok, 3570


#Training with CNN

In [ ]:
import tensorflow as tf
from tensorflow import keras

from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.utils import np_utils
import numpy as np
# 분류 대상 카테고리
root_dir = "/content/drive/My Drive/WikiExample/ch7/image/101_ObjectCategories"
categories = ["beaver", "panda", "Leopards", "dalmatian"]
nb_classes = len(categories)
image_size = 50
# 데이터 다운로드하기 --- (※1)
def main():
    X_train, X_test, y_train, y_test = np.load("/content/drive/My Drive/WikiExample/ch7/image/animals.npy",allow_pickle=True)
    # 데이터 정규화하기
    X_train = X_train.astype("float") / 256
    X_test  = X_test.astype("float")  / 256
    y_train = np_utils.to_categorical(y_train, nb_classes)
    y_test  = np_utils.to_categorical(y_test, nb_classes)
    # 모델을 훈련하고 평가하기
    model = model_train(X_train, y_train)
    model_eval(model, X_test, y_test)
# 모델 구축하기 --- (※2)
def build_model(in_shape):
    model = Sequential()
    model.add(Convolution2D(32, 3, 3, 
        border_mode='same',
        input_shape=in_shape))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Convolution2D(64, 3, 3, border_mode='same'))
    model.add(Activation('relu'))
    model.add(Convolution2D(64, 3, 3))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten()) 
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(nb_classes))
    model.add(Activation('softmax'))
    model.compile(loss='binary_crossentropy',
        optimizer='rmsprop',
        metrics=['accuracy'])
    return model
# 모델 훈련하기 --- (※3)
def model_train(X, y):
    model = build_model(X.shape[1:])
    model.fit(X, y, batch_size=32, nb_epoch=30)
    # 모델 저장하기 --- (※4)
    hdf5_file = "/content/drive/My Drive/WikiExample/ch7/image/animlas-model.hdf5"
    model.save_weights(hdf5_file)
    return model
# 모델 평가하기 --- (※5)
def model_eval(model, X, y):
    score = model.evaluate(X, y)
    print('loss=', score[0])
    print('accuracy=', score[1])

if __name__ == "__main__":
    main()

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(50, 50, 3..., padding="same")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:34: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), padding="same")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:36: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3))`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:52: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Epoch 1/30
3570/3570 [==============================] - 8s 2ms/step - loss: 0.4079 - accuracy: 0.8678
Epoch 2/30
3570/3570 [==============================] - 2s 557us/step - loss: 0.1056 - accuracy: 0.9612
Epoch 3/30
3570/3570 [==============================] - 2s 588us/step - loss: 0.0393 - accuracy: 0.9879
Epoch 4/30
3570/3570 [==============================] - 2s 598us/step - loss: 0.0319 - accuracy: 0.9907
Epoch 5/30
3570/3570 [==============================] - 2s 579us/step - loss: 0.0233 - accuracy: 0.9936
Epoch 6/30
3570/3570 [==============================] - 2s 571us/step - loss: 0.0179 - accuracy: 0.9955
Epoch 7/30
3570/3570 [==============================] - 2s 599us/step - loss: 0.0177 - accuracy: 0.9961
Epoch 8/30
3570/3570 [==============================] - 2s 555us/step - loss: 0.0207 - accuracy: 0.9954
Epoch 9/30
3570/3570 [==============================] - 2s 561us/step - loss: 0.0162 - accuracy: 0.9964
Epoch 10/30
3570/3570 [==============================] - 2s 570us/

#명령줄로 이미지 판정할 수 있게 하기

In [10]:
import os
os.chdir("/content/drive/My Drive/WikiExample/ch7/test")
#[출처] [구글 코랩] GOOGLE COLAB 파일 호출하기|작성자 러닝머신

In [11]:
import animals_keras as animals
import sys, os
from PIL import Image
import numpy as np
# 명령줄에서 파일 이름 지정하기 --- (※1)
if len(sys.argv) <= 1:
    print("animal-checker.py (<파일 이름>)")
    quit()
image_size = 50
categories = [
    "beaver", "panda", "Leopards", "dalmatian"]

# 입력 이미지를 Numpy로 변환하기 --- (※2)
X = []
files = []
for fname in sys.argv[1:]:
    img = Image.open(fname)
    img = img.convert("RGB")
    img = img.resize((image_size, image_size))
    in_data = np.asarray(img)
    X.append(in_data)
    files.append(fname)
X = np.array(X)
# CNN 모델 구축하기 --- (※3)
model = animals.build_model(X.shape[1:])
model.load_weights("/content/drive/My Drive/WikiExample/ch7/image/animlas-model.hdf5")
# 데이터 예측하기 --- (※4)
html = ""
pre = model.predict(X)
for i, p in enumerate(pre):
    y = p.argmax()
    print("+입력:", files[i])
    print("|동물 이름:", categories[y])
    html += """
        <h3>입력:{0}</h3>
        <div>
          <p><img src="{1}" width=300></p>
          <p>규동 이름:{2}</p>
        </div>
    """.format(os.path.basename(files[i]),
        files[i],
        categories[y])
# 리포트 저장하기 --- (※5)
html = "<html><body style='text-align:center;'>" + \
    "<style> p { margin:0; padding:0; } </style>" + \
    html + "</body></html>"
with open("animlas-result.html", "w") as f:
    f.write(html)

FileNotFoundError: ignored

#명령줄에서 실행

In [18]:
! python animals-checker.py pan.jpg dalma.jpg pan1.jpg beav.jpg leo.jpg beav1.jpg

2020-07-26 09:53:46.117413: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
Using TensorFlow backend.
/content/drive/My Drive/WikiExample/ch7/test/animals_keras.py:39: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(50, 50, 3..., padding="same")`
  input_shape=in_shape))
2020-07-26 09:53:48.705311: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-07-26 09:53:48.708312: E tensorflow/stream_executor/cuda/cuda_driver.cc:313] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2020-07-26 09:53:48.708353: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (028c735b5c0b): /proc/driver/nvidia/version does not exist
2020-07-26 09:53:48.713681: I tensorflow/core/platform/profile_utils/cpu_utils.cc:102] CPU Frequency: 230000